In [1]:
# import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# display dataset as a Pandas DataFrame

data = pd.read_csv('Resources/COVID_staff_shortages.csv')
data

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_admission_pediatric_covid_confirmed_5_11,previous_day_admission_pediatric_covid_confirmed_5_11_coverage,previous_day_admission_pediatric_covid_confirmed_unknown,previous_day_admission_pediatric_covid_confirmed_unknown_coverage,staffed_icu_pediatric_patients_confirmed_covid,staffed_icu_pediatric_patients_confirmed_covid_coverage,staffed_pediatric_icu_bed_occupancy,staffed_pediatric_icu_bed_occupancy_coverage,total_staffed_pediatric_icu_beds,total_staffed_pediatric_icu_beds_coverage
0,ME,2021/01/23,1,30,8,2,29,8,3.0,38,...,NaN,0,NaN,0,NaN,0,47.0,38,60.0,38
1,SD,2021/01/23,2,60,3,2,60,3,27.0,62,...,NaN,0,NaN,0,NaN,0,36.0,62,83.0,62
2,MD,2021/01/22,2,57,1,2,57,1,31.0,59,...,NaN,0,NaN,0,NaN,0,172.0,59,240.0,59
3,RI,2021/01/22,6,8,1,6,8,1,6.0,14,...,NaN,0,NaN,0,NaN,0,62.0,14,93.0,14
4,ID,2021/01/21,4,40,8,5,39,8,0.0,51,...,NaN,0,NaN,0,NaN,0,82.0,30,230.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54006,MT,2021/03/29,6,58,1,7,57,1,3.0,64,...,NaN,0,NaN,0,NaN,0,8.0,9,18.0,9
54007,NJ,2021/03/09,9,61,8,10,60,8,76.0,77,...,NaN,0,NaN,0,NaN,0,49.0,24,107.0,24
54008,NM,2021/05/19,12,43,1,16,39,1,2.0,55,...,NaN,0,NaN,0,NaN,0,0.0,19,0.0,19
54009,NE,2021/04/15,9,91,1,16,84,1,1.0,100,...,NaN,0,NaN,0,NaN,0,0.0,12,0.0,12


In [ ]:
# explore the data

# YOUR CODE HERE

In [7]:
#add columns for year, month, data for additional grouping/analysis 
data['date_for_split'] = data['date']
data[['year', 'month','day']] = data['date_for_split'].str.split('/', 2, expand=True)
data

#make df with select data of interest
select_df = data[['state','date','year','month','day',
                'critical_staffing_shortage_today_yes','critical_staffing_shortage_today_no','critical_staffing_shortage_today_not_reported',
                'inpatient_beds_used','inpatient_beds_used_covid',
                'previous_day_admission_adult_covid_confirmed',
                'previous_day_admission_pediatric_covid_confirmed',
                'staffed_adult_icu_bed_occupancy','staffed_adult_icu_bed_occupancy_coverage',
                'all_pediatric_inpatient_bed_occupied','all_pediatric_inpatient_bed_occupied_coverage',
                'previous_day_deaths_covid_and_influenza_coverage',]]
select_df

,state,date,year,month,day,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,inpatient_beds_used,inpatient_beds_used_covid,previous_day_admission_adult_covid_confirmed,previous_day_admission_pediatric_covid_confirmed,staffed_adult_icu_bed_occupancy,staffed_adult_icu_bed_occupancy_coverage,all_pediatric_inpatient_bed_occupied,all_pediatric_inpatient_bed_occupied_coverage,previous_day_deaths_covid_and_influenza_coverage
0,ME,2021/01/23,2021,01,23,1,30,8,2231.0,219.0,15.0,0.0,252.0,39,174.0,38,38
1,SD,2021/01/23,2021,01,23,2,60,3,1641.0,152.0,8.0,1.0,131.0,65,291.0,62,62
2,MD,2021/01/22,2021,01,22,2,57,1,9635.0,2039.0,217.0,0.0,1079.0,60,648.0,59,59
3,RI,2021/01/22,2021,01,22,6,8,1,2445.0,399.0,25.0,0.0,155.0,15,635.0,14,14
4,ID,2021/01/21,2021,01,21,4,40,8,2268.0,248.0,32.0,2.0,214.0,52,273.0,30,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54006,MT,2021/03/29,2021,03,29,6,58,1,1762.0,52.0,8.0,0.0,126.0,65,45.0,9,64
54007,NJ,2021/03/09,2021,03,09,9,61,8,15395.0,2008.0,241.0,4.0,1465.0,78,365.0,25,77
54008,NM,2021/05/19,2021,05,19,12,43,1,3181.0,163.0,21.0,0.0,355.0,56,36.0,19,55
54009,NE,2021/04/15,2021,04,15,9,91,1,2958.0,190.0,22.0,0.0,362.0,101,14.0,12,100


In [ ]:
"""Hypotheses
    Null 1: There is no relationshsip between pediatric and adult admittance throughout the year AND 
            they do not have repaltionship to staff shortages
    Altt 1: There is a relationship between adult and pediatric admittance throughout the year AND
            they do have a relationship to shatt shortages
    
    Null 2: there is no linear relationship between hospital staff shortages and adult or pediatric hospital admittance
    Altt 2: there is a linar relationship between shortages and hosital admittance 
    
    
    Null 3: There is no significant difference in the staff shortages through the months of the year and pediatric or adult admittance
    Altt 3: There is a significant difference in the staff shortages through the months of the year and pediatric or adult admittance
    

"""


In [ ]:
#group by dates
#states



In [ ]:
#plot data on x axis and 
#y axis is pediatic 
#y axis pediatic aduslt
#y axis shortage

#scatter plot pediatic and shortage
#scatter plot adult and shortage 